# Gemini API

The API (Application Programming Interface) of a large language model (LLM) refers to a set of predefined functions, protocols, and tools that allow developers to interact with the model programmatically. This interface enables external applications or systems to send prompts in form of texts or texts in combination with images, if the LLM is multimodal, specify criteria for generation such as temperature and system instructions, and receive generated outputs. The API essentially provides a way to integrate the LLM’s capabilities into other code.

Google provides a dedicated [GitHub Repository](https://github.com/google-gemini/cookbook) with code examples for using the API. The materials that we will see here are adapted from [this notebook](https://github.com/google-gemini/cookbook/blob/main/quickstarts/Get_started.ipynb).

## Getting an API Key

Gemini API has a free tier where users can register with a *personal gmail account* and obtain and API key. They don't have to enter any billing information. There are some limits in usage rates and models. But it is a great place to start exploring LLM APIs.

- Open [Google AI Studio](https://aistudio.google.com/app/u/1/prompts/new_chat)
  
<img src="../img/gemini_ai_studio.png" width="700"/>

- You will be prompted to log in. Please log in with your **personal Google account**.
- If you are logged in with your institutional account, you might see that you are not allowed by the organization admin to use these services.

<img src="../img/gemini_ai_studio_institutional.png" width="700"/>

- Click `Get API key` on the upper right corner.

<img src="../img/gemini_ai_studio_api.png" width="700"/>

- Click `Create API key` on the right corner below the navigation bar. If you are doing this for the first time, you should see a pop up that asks you to agree to the terms of use.

<img src="../img/gemini_ai_studio_consent.png" width="500"/>

- After you agree, you should see another pop up that allows you to create an API key in an existing project or in a new one if you do not have any projects on Google Cloud Services.

<img src="../img/gemini_ai_studio_create_api.png" width="400"/>

- Your API key should appear on the screen. Please copy this key for the next steps.

<img src="../img/gemini_ai_studio_api_key.png" width="600"/>



## API keys and .env files

In lesson 4-3_api_NYT.ipynb we saw one way to safely use an API key in a Jupyter Notebook. Another way to do this is using `.env` files. This is particularly useful when you are coding in a repository where you might accidently push your API secret keys to GitHub. Or if you ever want to deploy your code, you will need to deal with `.env` files.

- Create a new file in your repository called `.env` This is the entire filename. Leave it empty for now.
- Create another file called `.gitignore`.
  You can add anything in the `.gitignore` file that you want GitHub to not track. This will prevent you from pushing files that you don't want changed.
- On an empty line in the  `.gitignore` file, type `.env` and push the changes.
  This tells GitHub to ignore the `.env` file going forward.
- Now you can save your API key to your `.env` file. Open your `.env` file and type `GEMINI_API_KEY="YOUR_API_KEY"`
- Save your `.env` file and close it. Make sure that .env file is not listed in the source control section.
  
API keys, other secrets as well as their management are really important skills to learn. A great place to start reading more about this topic is on this [blog post](https://dev.to/jakewitcher/using-env-files-for-environment-variables-in-python-applications-55a1). While preparing these materials, I found a new [VSCode extension](https://marketplace.visualstudio.com/items?itemName=josephdavidwilsonjr.api-vault), which I have not tested yet so I cannot recommend or urge against the use of it. If you want, you can take a look at it [here](https://medium.com/@dingersandks/why-every-developers-api-keys-are-probably-in-the-wrong-place-and-how-a-vs-code-extension-finally-c966d081d132). If for nothing else, this post shows why API keys are so important.

In [1]:
# Let's load our API key from the environment
import os
from dotenv import load_dotenv

load_dotenv() # By default load_dotenv will look for the .env file in the current working directory

api_key = os.getenv("GEMINI_API_KEY")

if api_key:
    print("API key loaded successfully.")

API key loaded successfully.
